# Import das bibliotecas

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Leitura dos dados

In [29]:
data_training = pd.read_csv('falencia-treinamento.csv', delimiter=';')
data_test = pd.read_csv('falencia-teste.csv', delimiter=';')
data_result = pd.read_csv('modelo-submissao.csv', delimiter=';')

# Pré-Processamento dos dados

In [30]:
X_train, X_test, y_train, y_test = train_test_split(data_training.drop('Resultado', axis=1), data_training['Resultado'], test_size=0.2, random_state=42)

# Regressão Logística

In [25]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=500)
logistic_model.fit(X_train, y_train)

y_pred = logistic_model.predict(X_test)

# Calcula a acurácia do modelo nos dados de teste extraído dos de treinamento
accuracy = accuracy_score(y_test, y_pred)
f_measure = f1_score(y_test, y_pred)

# resultados
print(f'A acurácia do modelo de regressão logística é de {accuracy}')
print(f"A F-measure do modelo de regressão logística é de {f_measure}")

A acurácia do modelo de regressão logística é de 0.7375
A F-measure do modelo de regressão logística é de 0.43243243243243246


# Árvore de Decisão

In [26]:
from sklearn.tree import DecisionTreeClassifier

# criar um modelo de árvore de decisão com profundidade máxima de 3
tree_model = DecisionTreeClassifier(max_depth=3)

# ajustar o modelo aos dados de treinamento
tree_model.fit(X_train, y_train)

# fazer previsões nos dados de teste
y_pred = tree_model.predict(X_test)

# calcular a acurácia e f-measure do modelo nos dados de teste
accuracy = accuracy_score(y_test, y_pred)
f_measure = f1_score(y_test, y_pred)

# resultados
print(f"A acurácia do modelo de árvore de decisão é de {accuracy}")
print(f"A F-measure do modelo de árvore de decisão é de {f_measure}")

A acurácia do modelo de árvore de decisão é de 0.725
A F-measure do modelo de árvore de decisão é de 0.5217391304347826


# Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# criar um modelo de Random Forest com 100 árvores de decisão
rf_model = RandomForestClassifier(n_estimators=100)

# ajustar o modelo aos dados de treinamento
rf_model.fit(X_train, y_train)

# fazer previsões nos dados de teste
y_pred = rf_model.predict(X_test)

# calcular a acurácia e f-measure do modelo nos dados de teste
accuracy = accuracy_score(y_test, y_pred)
f_measure = f1_score(y_test, y_pred)

# resultados
print(f"A acurácia do modelo de Random Forest é de {accuracy}")
print(f"A F-measure do modelo de Random Forest é de {f_measure}")

A acurácia do modelo de Random Forest é de 0.75
A F-measure do modelo de Random Forest é de 0.5454545454545454


# Comparação entre os modelos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# calcular a acurácia e F-measure de cada modelo
accuracy_lr, f_measure_lr = accuracy_score(y_test, y_pred_lr), f1_score(y_test, y_pred_lr)
accuracy_dt, f_measure_dt = accuracy_score(y_test, y_pred_dt), f1_score(y_test, y_pred_dt)
accuracy_rf, f_measure_rf = accuracy_score(y_test, y_pred_rf), f1_score(y_test, y_pred_rf)

# definir os rótulos do eixo x e os valores do eixo y
labels = ['Regressão Logística', 'Árvore de Decisão', 'Random Forest']
accuracy_values = [accuracy_lr, accuracy_dt, accuracy_rf]
f_measure_values = [f_measure_lr, f_measure_dt, f_measure_rf]

# definir a largura das barras
bar_width = 0.35

# definir a posição das barras no eixo x
x = np.arange(len(labels))

# criar o gráfico de barras
fig, ax = plt.subplots()
rects1 = ax.bar(x - bar_width/2, accuracy_values, bar_width, label='Acurácia')
rects2 = ax.bar(x + bar_width/2, f_measure_values, bar_width, label='F-measure')

# adicionar rótulos, títulos e legendas ao gráfico
ax.set_ylabel('Métrica')
ax.set_title('Comparação de métricas entre modelos')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# exibir o gráfico
plt.show()